In [1]:
from translate import Translator
import geopandas as gpd
import pandas as pd

In [2]:
from pandas.errors import EmptyDataError

def export_dict_csv(d, path, column_names):
    """
    Exportuje slovník ako CSV vo formáte kľúč, hodnota.

    Argumenty:
    d (dict): Slovník, ktorý sa má exportovať.
    path (str): Cesta k súboru, kde sa CSV uloží.
    column_names (list): Zoznam názvov stĺpcov pre CSV súbor. Môžu byť maximálne dva názvy.

    Návratová hodnota:
    None

    Výnimky:
    Exception: Ak je v column_names viac ako 2 názvy stĺpcov.

    Príklad použitia:
    >>> d = {'hello': 'ahoj', 'world': 'svet'}
    >>> export_dict_csv(d, 'output.csv', ['EN', 'SK'])
    """
    if len(column_names) > 2:
        raise Exception('Príliš veľa názvov stĺpcov. Môžu byť maximálne 2.')
    
    # Vytvorenie DataFrame zo slovníka a nastavenie názvov stĺpcov
    df = pd.DataFrame(list(d.items()), columns=column_names)
    
    # Exportovanie DataFrame do CSV
    df.to_csv(path, index=False)

def csv_to_dict(path, key_column, value_column):
    """
    Načíta CSV ako slovník.

    Argumenty:
    path (str): Cesta k CSV súboru, ktorý sa má načítať.
    key_column (str): Názov stĺpca, ktorý bude použitý ako kľúč v slovníku.
    value_column (str): Názov stĺpca, ktorý bude použitý ako hodnota v slovníku.

    Návratová hodnota:
    dict: Načítaný slovník.

    Výnimky:
    EmptyDataError: Ak je CSV súbor prázdny.

    Príklad použitia:
    >>> d = csv_to_dict('output.csv', 'EN', 'SK')
    """
    try:
        # Načítanie CSV do DataFrame
        df = pd.read_csv(path)
        
        # Vytvorenie slovníka zo stĺpcov DataFrame
        d = pd.Series(df[value_column].values, index=df[key_column]).to_dict()
    except pd.errors.EmptyDataError:
        return dict()
    
    return d


NAČÍTANIE DÁT

In [3]:
data = gpd.read_file(r'./source-files/2_doplnena_obec.geojson')
translator = Translator(to_lang="sk")

In [4]:
data.columns

Index(['index', 'id', 'abandoned', 'access', 'addr:postcode', 'addr:street',
       'addr:streetnumber', 'amenity', 'building', 'contact:instagram',
       'contact:tiktok', 'contact:youtube', 'drinking_water', 'height',
       'hoops', 'internet_access', 'landuse', 'leisure', 'leisure_1', 'length',
       'lit', 'location', 'natural', 'net', 'opening_hours', 'operator',
       'operator:type', 'owner', 'reservation', 'seasonal', 'source', 'sport',
       'start_date', 'supervised', 'surface', 'swimming_lanes',
       'swimming_pool', 'toilets:wheelchair', 'water', 'water:taste',
       'water_characteristic', 'wheelchair', 'width', '@timestamp',
       'vsetky_sporty_str', 'kontakt_email', 'kontakt_facebook',
       'kontakt_telefon', 'kontakt_webstranka', 'max_hlbka', 'min_hlbka',
       'hlbka', 'teplota_vody', 'nazov', 'max_teplota', 'kapacita_divaci',
       'poplatok', 'typ', 'futbal', 'tenis', 'basketbal', 'hokej', 'volejbal',
       'plavanie', 'viacucelove', 'zdroj', 'status',

PREKLAD ZÁZNAMOV

In [5]:
to_translate = [
    'abandoned',
    'access',
    'amenity',
    'building',
    'drinking_water',
    'hoops',
    'internet_access',
    'lit',
    'location',
    'natural',
    'net',
    'operator:type',
    'reservation',
    'seasonal',
    'supervised',
    'surface',
    'swimming_pool',
    'toilets:wheelchair',
    'water',
    'water_characteristic',
    'wheelchair',
    'poplatok'
]


manual_translations = {'private': 'Súkromné', 'yes': 'Áno', 'no': 'Nie', 'grass': 'Tráva', 'clay': 'Antuka', 'the surface is clay': 'Antuka', 'indoor': 'Indoor', 'outdoor': 'Outdoor', 'pebblestone': 'Kamienky', 'dirt': 'Hlina', 'box': 'Box', 'field_hockey': 'Hokejbal', 'lake': 'Jazero', 'Lake': 'Jazero'}
saved_translations = csv_to_dict(r'source-files\anglicko_slovenske_preklady.csv', 'EN', 'SK')
translations = manual_translations | saved_translations # spoji slovniky s ulozenymi prekladmi a manualnymi prekladmi, translations['anglicky vyraz'] = 'slovensky preklad'

In [6]:
# v kazdom stlpci ku prelozeniu zisti unikatne hodnoty, ktore ak sa uz ich preklady nenachadzaju v translations slovniku, tak prelozi a priradi do translations slovniku
for column_name in to_translate:
    
    unique_values = data[column_name].unique()
        
    for val in unique_values:
        if not val or pd.isna(val):
            continue
        val = val.lower()
        val = val.replace('_', ' ')
        if val not in translations.keys():
            translations[val] = translator.translate(val.capitalize())
        
    data[column_name] = data[column_name].map(translations)


PREKLAD NÁZVOV ATRIBÚTOV

In [7]:
column_name_translation = {
    'abandoned': 'opustene',
    'access': 'pristup',
    'addr:postcode': 'adresa_PSC',
    'addr:street': 'adresa_ulica',
    'addr:streetnumber': 'adresa_cislodomu',
    'amenity': 'zariadenie',
    'building': 'budova',
    'contact:instagram': 'kontakt_instagram',
    'contact:tiktok': 'kontakt_tiktok',
    'contact:youtube': 'kontakt_youtube',
    'depth': 'hlbka',
    'drinking_water': 'pitna_voda',
    'height': 'vyska',
    'hoops': 'basketbalove_kose',
    'internet_access': 'internet_pristup',
    'leisure': 'osm_leisure',
    'leisure_1': 'osm_leisure1',
    'landuse': 'osm_landuse',
    'length': 'dlzka',
    'lit': 'umele_osvetlenie',
    'location': 'umiestnenie',
    'natural': 'prirodne',
    'net': 'siet',
    'opening_hours': 'otvaracie_hodiny',
    'operator': 'prevadzkovatel',
    'operator:type': 'typ_prevadzkovatel',
    'outdoor': 'outdoor',
    'owner': 'majitel',
    'reservation': 'rezervacia',
    'seasonal': 'sezonne',
    'source': 'osm_zdroj',
    'sport': 'osm_sport',
    'start_date': 'd_zaciatok',
    'supervised': 'pod_dohladom',
    'surface': 'povrch',
    'swimming_lanes': 'plavecke_pruhy',
    'swimming_pool': 'bazen',
    'temperature': 'teplota',
    'toilets:wheelchair': 'zachody_invalid',
    'water': 'voda',
    'water:taste': 'chut_vody',
    'water_characteristic': 'charakter_vody',
    'wheelchair': 'invalid',
    'width': 'sirka',
    '@timestamp': 'naposledy_upravene'
}

data.rename(columns=column_name_translation, inplace=True)

PREKLAD OTVÁRACÍCH HODÍN

In [8]:
def translate_opening_hours(opening_hours):
    """
    Preloží otváracie hodiny zo skrátených anglických názvov dní na slovenské.

    Argumenty:
    opening_hours (str): Reťazec obsahujúci otváracie hodiny.

    Návratová hodnota:
    str: Preložené otváracie hodiny, alebo None, ak bol vstup None.

    Príklad použitia:
    >>> translate_opening_hours("Mo-Fr 08:00-16:00")
    'Po-Pi 08:00-16:00'
    """

    def map_substrings(input_string, mapping_dict):
        """
        Premapuje substringy v reťazci na iný reťazec pomocou mapovacieho slovníka.

        Argumenty:
        input_string (str): Vstupný reťazec, ktorý sa má premapovať.
        mapping_dict (dict): Slovník obsahujúci mapovania vo forme {substring: náhrada}.

        Návratová hodnota:
        str: Reťazec s premapovanými substringami.

        Príklad použitia:
        >>> map_substrings("mo-fr", {'mo': 'Po', 'fr': 'Pi'})
        'Po-Pi'
        """
        
        # Zoradí kľúče slovníka podľa dĺžky v zostupnom poradí
        sorted_keys = sorted(mapping_dict.keys(), key=len, reverse=True)
    
        # Pre každý kľúč v zoradených kľúčoch nahradí všetky výskyty v vstupnom reťazci
        for key in sorted_keys:
            if key in input_string:
                input_string = input_string.replace(key, mapping_dict[key])
    
        return input_string
    
    # Slovník s prekladmi skrátených názvov dní
    translations = {
        'mo': 'Po',
        'tu': 'Ut',
        'we': 'St',
        'th': 'Št',
        'fr': 'Pi',
        'sa': 'So',
        'su': 'Ne',
        'oct': 'okt',
        'may': 'máj'
    }
    
    # Preklad reťazca otváracích hodín pomocou map_substrings alebo vrátenie None, ak je vstup None
    return map_substrings(opening_hours.lower(), translations) if opening_hours else None


data['otvaracie_hodiny'] = data['otvaracie_hodiny'].apply(translate_opening_hours)

PREKLAD ŠPORTOV

In [9]:
data['vsetky_sporty_str'].unique()

array(['', 'basketball', 'swimming', 'soccer', 'multi', 'tennis',
       'volleyball', 'ice_hockey;ice_skating', 'ice_hockey',
       'ice_skating', 'field_hockey', 'beachvolleyball', 'hockey',
       'soccer;tennis', 'ice_hockey;skating', 'soccer;ice_hockey',
       'football;multi', 'soccer;basketball', 'street_hockey',
       'ice_hockey;ball_hockey;ice_skating'], dtype=object)

In [10]:
# Slovník na preklad športov z anglického do slovenského jazyka
sport_map_dict = {
    'basketball': 'Basketbal',
    'swimming': 'Plávanie',
    'tennis': 'Tenis',
    'ice_hockey': 'Hokej',
    'ice_skating': 'Ľadové korčuľovanie',
    'soccer': 'Futbal',
    'hockey': 'Hokej',
    'field_hockey': 'Hokejbal',
    'multi': 'Viacúčelové',
    'skating': 'Korčuľovanie',
    'street_hockey': 'Hokejbal',
    'volleyball': 'Volejbal',
    'beachvolleyball': 'Plážový volejbal',
    'football': 'Futbal',
    'ball_hockey': 'Hokejbal'
}

def translate_sport(value):
    """
    Preloží športy v reťazci oddelenom bodkočiarkou pomocou mapovacieho slovníka.

    Argumenty:
    value (str): Reťazec športov oddelený bodkočiarkou.

    Návratová hodnota:
    str: Preložený reťazec športov oddelený čiarkou.

    Príklad použitia:
    >>> translate_sport('soccer;basketball;swimming')
    'Futbal, Basketbal, Plávanie'
    """
    # Rozdelí vstupný reťazec na jednotlivé športy
    elements = value.split(';')
    
    # Preloží každý šport pomocou sport_map_dict, ak preklad neexistuje, použije pôvodný názov
    translated_elements = [sport_map_dict.get(element, element) for element in elements]
    
    # Spojí preložené športy do jedného reťazca oddeleného čiarkou
    return ', '.join(translated_elements)

# Aplikuje funkciu translate_sport na stĺpec 'vsetky_sporty_str' v DataFrame 'data'
data['vsetky_sporty_str'] = data['vsetky_sporty_str'].apply(translate_sport)

# Zobrazí unikátne hodnoty v stĺpci 'vsetky_sporty_str' po preklade
data['vsetky_sporty_str'].unique()


array(['', 'Basketbal', 'Plávanie', 'Futbal', 'Viacúčelové', 'Tenis',
       'Volejbal', 'Hokej, Ľadové korčuľovanie', 'Hokej',
       'Ľadové korčuľovanie', 'Hokejbal', 'Plážový volejbal',
       'Futbal, Tenis', 'Hokej, Korčuľovanie', 'Futbal, Hokej',
       'Futbal, Viacúčelové', 'Futbal, Basketbal',
       'Hokej, Hokejbal, Ľadové korčuľovanie'], dtype=object)

In [11]:
data

,index,id,opustene,pristup,adresa_PSC,adresa_ulica,adresa_cislodomu,zariadenie,budova,kontakt_instagram,...,hokej,volejbal,plavanie,viacucelove,zdroj,status,obec_id,okres_id,kraj_id,geometry
0,11558,node/10003968772,NaN,NaN,08001,Škultétyho,None,NaN,NaN,None,...,0,0,1,0,OpenStreetMap,Aktívne,1823,48.0,5.0,POINT (21.24419 48.98785)
1,11559,node/10084161428,NaN,Áno,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2920,79.0,8.0,POINT (18.74262 49.21629)
2,11560,node/10084161430,NaN,Áno,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2920,79.0,8.0,POINT (18.73872 49.21501)
3,11561,node/10159452050,NaN,Áno,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2279,65.0,4.0,POINT (18.17113 48.55325)
4,11562,node/10253818042,NaN,NaN,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,155,4.0,2.0,POINT (17.11560 48.15653)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12419,8611,way/998690024,NaN,NaN,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2886,32.0,1.0,"POLYGON ((19.81850 48.30302, 19.81889 48.30256..."
12420,8612,way/999288148,NaN,NaN,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2303,67.0,6.0,"POLYGON ((18.16784 48.90731, 18.16819 48.90732..."
12421,8613,way/999288152,NaN,NaN,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2303,67.0,6.0,"POLYGON ((18.16776 48.90543, 18.16800 48.90537..."
12422,8614,way/999288153,NaN,NaN,None,None,None,NaN,NaN,None,...,0,0,0,0,OpenStreetMap,Aktívne,2303,67.0,6.0,"POLYGON ((18.16804 48.90597, 18.16787 48.90601..."


EXPORT

In [12]:
# slovnik translations sa exportuje ako csv a pri dalsom behu procesu sa pouzije k prekladu, miesto prekladania cez kniznicu translate, co usetri cas
export_dict_csv(translations, r'./source-files/anglicko_slovenske_preklady.csv', ['EN', 'SK'])

In [13]:
data.to_file(r'./source-files/3_prelozene.geojson', driver = 'GeoJSON')